# Download Data - CBSA

In [10]:
#import functions
import os
import sys

sys.path.append(
    os.path.join(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
)

#censusdis
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt

from typing import Optional

import censusdis.data as ced
import censusdis.maps as cem
import censusdis.values as cev
import censusdis.geography as cgeo
from censusdis.states import STATE_MA
from censusdis import states
from censusdis.maps import ShapeReader, plot_us_boundary
import censusdis.maps as cmap


# Make sure it is there.
from censusdis.values import ALL_SPECIAL_VALUES

# Leafmap
from ipyleaflet import *
import leafmap

# _______________________________________________________________________

#standard packages
import pandas as pd
import numpy as np
import math
from math import pi, sqrt
import matplotlib.pyplot as plt
# import pygwalker as pyg

# import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import datetime
import time
# from tqdm import tqdm, trange

#gis packages
import osmnx as ox
import logging as lg
ox.settings.log_console=True #use cache to avoid overloading the server
# ox.settings.memory_cache=True #use cache to avoid overloading the server
ox.settings.memory = 4294967296 #set memory cache to 4GB

from shapely.geometry import Point
import folium
import networkx as nx
## future libaries
# import contextily as cx
# import fiona
# from pandana.loaders import osm
# import momepy
# import missingno as msno
# from us import states
# import imageio

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
ox.__version__

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)

# Population Demographics

In [11]:
# Data information
YEAR = 2022 # The year we want data for.
GEOYEAR = 2021 # The year we want geography for.
DATASET = "acs/acs5" # Dataset
api_key = "781655d3b92f252d48d8acae0f5669dddf3cb9a4" # API key

In [12]:
# download shell file and keep only General Model rows

acs22_5_shell_genmodel = pd.read_excel('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/ACS20225YR_Table_Shells.xlsx', sheet_name='ACS20225YR_Table_Shells')

acs22_5_shell_genmodel.head()

,Index,Table_ID,Column1,Census_ACS,Category,Race,Line,Indent,Unique_ID,Label,Sex,Theme,Title,Universe,General_Model,General_Model_Broad,Sex Check,Type
0,1,B01001,B01001,B,1001,NaN,1,0,B01001_001,Total:,Both,Age and Sex,Sex by Age,Total population,1.0,1.0,True,int
1,2,B01001,B01001,B,1001,NaN,2,1,B01001_002,Male:,Male,Age and Sex,Sex by Age,Total population,1.0,1.0,True,int
2,3,B01001,B01001,B,1001,NaN,3,2,B01001_003,Under 5 years,Male,Age and Sex,Sex by Age,Total population,1.0,1.0,True,int
3,4,B01001,B01001,B,1001,NaN,4,2,B01001_004,5 to 9 years,Male,Age and Sex,Sex by Age,Total population,1.0,1.0,True,int
4,5,B01001,B01001,B,1001,NaN,5,2,B01001_005,10 to 14 years,Male,Age and Sex,Sex by Age,Total population,1.0,1.0,True,int


In [13]:
#ger varialbes for general model

variables = acs22_5_shell_genmodel[acs22_5_shell_genmodel['General_Model'] == 1][['Table_ID']].drop_duplicates()
variable_list = variables['Table_ID'].tolist()
variable_list =variable_list[:] #test with 5 variables
variable_list

['B01001',
 'B01002',
 'B01003',
 'B02001',
 'B08103',
 'B08135',
 'B08301',
 'B08303',
 'B08603',
 'B08604',
 'B09001',
 'B09020',
 'B11001',
 'B11003',
 'B11007',
 'B13016',
 'B14003',
 'B15001',
 'B15011',
 'B15012',
 'B17001',
 'B17003',
 'B17006',
 'B17010',
 'B17020',
 'B18101',
 'B19001',
 'B19013',
 'B19019',
 'B19025',
 'B19037',
 'B19050',
 'B19051',
 'B19052',
 'B19053',
 'B19054',
 'B19055',
 'B19056',
 'B19057',
 'B19058',
 'B19059',
 'B19060',
 'B19061',
 'B19062',
 'B19063',
 'B19064',
 'B19065',
 'B19066',
 'B19067',
 'B19069',
 'B19070',
 'B19083',
 'B19101',
 'B19113',
 'B19119',
 'B19126',
 'B19127',
 'B19201',
 'B19313',
 'B20001',
 'B20002',
 'B20004',
 'B20005',
 'B21001',
 'B21004',
 'B22001',
 'B22002',
 'B22003',
 'B23001',
 'B24012',
 'B24032',
 'B24080',
 'B24082',
 'B24115',
 'B25001',
 'B25002',
 'B25003',
 'B25009',
 'B25010',
 'B25013',
 'B25017',
 'B25018',
 'B25019',
 'B25020',
 'B25021',
 'B25022',
 'B25024',
 'B25031',
 'B25034',
 'B25035',
 'B25036',

In [ ]:
# code cursor

dataset = None

for i, variable in enumerate(variable_list):
    print(f"{i}, {variable}")
    file_path = f'C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/5YRData/acsdt5y2022-{variable}.dat'
    
    if os.path.exists(file_path):
        try:
            data = pd.read_csv(file_path, sep='|', low_memory=False)
            if 'GEO_ID' in data.columns:
                if dataset is None:
                    dataset = data
                else:
                    # Use outer merge to keep all rows from both dataframes
                    dataset = pd.merge(dataset, data, on='GEO_ID', how='outer')
                print(f"Merged {variable}, new shape: {dataset.shape}")
            else:
                print(f"Warning: 'GEO_ID' not found in {variable}")
        except Exception as e:
            print(f"Error reading file for {variable}: {str(e)}")
    else:
        print(f"File not found: {file_path}")

if dataset is not None:
    print("Final dataset shape:", dataset.shape)
else:
    print("No valid dataframes to merge.")

In [22]:
import os
import pandas as pd
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Function to read CSV efficiently
def read_csv_efficient(file_path):
    # First, read just the header to get column names
    header = pd.read_csv(file_path, sep='|', nrows=0)
    columns = ['GEO_ID'] + [col for col in header.columns if col.endswith('E001')]
    
    # Then read the actual data with selected columns
    return dd.read_csv(file_path, sep='|', usecols=columns, 
                       engine='pyarrow', assume_missing=True)

# Initialize empty list to store dataframes
dfs = []

# Read all files in parallel
with ProgressBar():
    dfs = [
        read_csv_efficient(f'C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/5YRData/acsdt5y2022-{variable}.dat')
        for variable in variable_list
        if os.path.exists(f'C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/5YRData/acsdt5y2022-{variable}.dat')
    ]

# Merge all dataframes
if dfs:
    with ProgressBar():
        # Compute each dataframe and set index
        computed_dfs = [df.set_index('GEO_ID').compute() for df in dfs]
        
        # Use pandas to concatenate
        dataset = pd.concat(computed_dfs, axis=1, join='outer')
    
    print("Final dataset shape:", dataset.shape)
else:
    print("No valid dataframes to merge.")

[########################################] | 100% Completed | 1.29 sms
[########################################] | 100% Completed | 590.46 ms
[########################################] | 100% Completed | 361.54 ms
[########################################] | 100% Completed | 358.31 ms
[########################################] | 100% Completed | 457.15 ms
[########################################] | 100% Completed | 283.53 ms
[########################################] | 100% Completed | 231.23 ms
[########################################] | 100% Completed | 516.97 ms
[########################################] | 100% Completed | 447.49 ms
[########################################] | 100% Completed | 225.22 ms
[########################################] | 100% Completed | 271.26 ms
[########################################] | 100% Completed | 280.56 ms
[########################################] | 100% Completed | 469.24 ms
[########################################] | 100% Completed | 396

In [23]:
dataset

,B01001_E001,B01002_E001,B01003_E001,B02001_E001,B08103_E001,B08135_E001,B08301_E001,B08303_E001,B08603_E001,B08604_E001,B09001_E001,B09020_E001,B11001_E001,B11003_E001,B11007_E001,B13016_E001,B14003_E001,B15001_E001,B15011_E001,B15012_E001,B17001_E001,B17003_E001,B17006_E001,B17010_E001,B17020_E001,B18101_E001,B19001_E001,B19013_E001,B19019_E001,B19025_E001,B19037_E001,B19050_E001,B19051_E001,B19052_E001,B19053_E001,B19054_E001,B19055_E001,B19056_E001,B19057_E001,B19058_E001,B19059_E001,B19060_E001,B19061_E001,B19062_E001,B19063_E001,B19064_E001,B19065_E001,B19066_E001,B19067_E001,B19069_E001,B19070_E001,B19083_E001,B19101_E001,B19113_E001,B19119_E001,B19126_E001,B19127_E001,B19201_E001,B19313_E001,B20001_E001,B20002_E001,B20004_E001,B20005_E001,B21001_E001,B21004_E001,B22001_E001,B22002_E001,B22003_E001,B23001_E001,B24012_E001,B24032_E001,B24080_E001,B24082_E001,B24115_E001,B25001_E001,B25002_E001,B25003_E001,B25009_E001,B25010_E001,B25013_E001,B25017_E001,B25018_E001,B25019_E001,B25020_E001,B25021_E001,B25022_E001,B25024_E001,B25031_E001,B25034_E001,B25035_E001,B25036_E001,B25037_E001,B25040_E001,B25041_E001,B25042_E001,B25056_E001,B25075_E001,B25085_E001,B25086_E001,B25104_E001,B25105_E001,B25118_E001,B25119_E001,B25120_E001,B25132_E001,B25133_E001,B25134_E001,B25135_E001,B27001_E001,B27002_E001,B27003_E001
GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0100000US,331097593.0,38.5,331097593.0,331097593.0,41.6,3.689928e+09,156703623.0,138386938.0,NaN,NaN,73213705.0,54737648.0,125736353.0,81432908.0,125736353.0,77210485.0,320051509.0,257883888.0,77751347.0,83932731.0,323275448.0,223101412.0,71730273.0,81432908.0,323275448.0,326147510.0,125736353.0,75149.0,75149.0,1.330706e+13,125736353.0,1.330706e+13,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,1.051612e+13,9.880560e+12,6.359560e+11,6.750147e+11,8.908618e+11,7.191745e+10,1.416723e+10,9.321670e+11,2.064164e+11,0.4829,81432908.0,92646.0,92646.0,92646.0,1.014083e+13,44303445.0,1.366157e+13,176037501.0,42131.0,48747.0,266411973.0,256649167.0,38374.0,125736353.0,125736353.0,125736353.0,266411973.0,45768.0,53073.0,158913204.0,53073.0,83957980.0,140943613.0,140943613.0,125736353.0,125736353.0,2.57,125736353.0,140943613.0,5.5,820090169.0,125736353.0,5.7,744990135.0,140943613.0,1268.0,140943613.0,1979.0,125736353.0,1979.0,125736353.0,140943613.0,125736353.0,44238593.0,81497760.0,1549548.0,6.192507e+11,125736353.0,1276.0,125736353.0,75149.0,1.330706e+13,125736353.0,125736353.0,125736353.0,125736353.0,326147510.0,326147510.0,326147510.0
0100089US,1072026.0,36.6,1072026.0,1072026.0,42.9,8.488515e+06,393272.0,359059.0,NaN,NaN,278468.0,174653.0,361167.0,243943.0,361167.0,235624.0,1032901.0,793558.0,127372.0,136976.0,1056357.0,683859.0,271310.0,243943.0,1056357.0,1062680.0,361167.0,54612.0,54612.0,2.732731e+10,361167.0,2.732731e+10,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,1.968512e+10,1.822880e+10,1.456456e+09,1.622007e+09,2.662245e+09,3.173004e+08,6.238790e+07,2.235395e+09,7.427111e+08,0.4870,243943.0,64684.0,64684.0,64684.0,2.099607e+10,117224.0,2.884370e+10,459371.0,34205.0,38453.0,826717.0,792603.0,29624.0,361167.0,361167.0,361167.0,826717.0,37188.0,41891.0,401410.0,41891.0,NaN,450598.0,450598.0,361167.0,361167.0,2.93,361167.0,450598.0,5.2,2426839.0,361167.0,5.3,2030095.0,450598.0,763.0,450598.0,1983.0,361167.0,1983.0,361167.0,450598.0,361167.0,105263.0,255904.0,3248.0,1.048378e+09,361167.0,659.0,361167.0,54612.0,2.732731e+10,361167.0,361167.0,361167.0,361167.0,1062680.0,1062680.0,1062680.0
0100090US,1171.0,46.9,1171.0,1171.0,43.8,8.250000e+03,396.0,371.0,NaN,NaN,250.0,274.0,487.0,336.0,487.0,228.0,1123.0,921.0,118.0,118.0,1171.0,848.0,250.0,336.0,1171.0,1171.0,487.0,44438.0,44438.0,3.343570e+07,487.0,NaN,487.0,487.0,487.0,487.0,487.0,487.0,487.0,487.0,487.0,487.0,2.487350e+07,2.440830e+07,4.652

# Import Geography file


In [33]:
# Import Geography

geography = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/Geos20225YR.txt', sep='|')
geography.head(2)

,FILEID,STUSAB,SUMLEVEL,COMPONENT,US,REGION,DIVISION,STATE,COUNTY,COUSUB,PLACE,TRACT,BLKGRP,CONCIT,AIANHH,AIANHHFP,AIHHTLI,AITS,AITSFP,ANRC,CBSA,CSA,METDIV,MACC,MEMI,NECTA,CNECTA,NECTADIV,UA,CDCURR,SLDU,SLDL,ZCTA5,SUBMCD,SDELM,SDSEC,SDUNI,UR,PCI,PUMA5,GEO_ID,NAME,BTTR,BTBG,TL_GEO_ID
0,ACSSF,US,10,00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100000US,United States,NaN,NaN,NaN
1,ACSSF,US,10,89,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100089US,United States -- American Indian Reservation and Trust Land -- Federal,NaN,NaN,NaN


In [34]:
geography.columns

Index(['FILEID', 'STUSAB', 'SUMLEVEL', 'COMPONENT', 'US', 'REGION', 'DIVISION',
       'STATE', 'COUNTY', 'COUSUB', 'PLACE', 'TRACT', 'BLKGRP', 'CONCIT',
       'AIANHH', 'AIANHHFP', 'AIHHTLI', 'AITS', 'AITSFP', 'ANRC', 'CBSA',
       'CSA', 'METDIV', 'MACC', 'MEMI', 'NECTA', 'CNECTA', 'NECTADIV', 'UA',
       'CDCURR', 'SLDU', 'SLDL', 'ZCTA5', 'SUBMCD', 'SDELM', 'SDSEC', 'SDUNI',
       'UR', 'PCI', 'PUMA5', 'GEO_ID', 'NAME', 'BTTR', 'BTBG', 'TL_GEO_ID'],
      dtype='object')

In [35]:

# Select Geography Columns

# geography = geography[geography['BLKGRP'] > 1] # columns where there are blockgroup information
geography = geography[[
    'FILEID',
    'US',
    'REGION',
    'DIVISION',
    'STATE',
    'COUNTY',
    'COUSUB',
    'PLACE',
    'TRACT',
    'BLKGRP',
    'CBSA',
    'CSA',
    'METDIV',
    'CDCURR',
    'GEO_ID',
    'NAME',
    'TL_GEO_ID'
]]
geography.head()

,FILEID,US,REGION,DIVISION,STATE,COUNTY,COUSUB,PLACE,TRACT,BLKGRP,CBSA,CSA,METDIV,CDCURR,GEO_ID,NAME,TL_GEO_ID
0,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100000US,United States,NaN
1,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100089US,United States -- American Indian Reservation and Trust Land -- Federal,NaN
2,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100090US,United States -- American Indian Reservation and Trust Land -- State,NaN
3,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100091US,United States -- Oklahoma Tribal Statistical Area,NaN
4,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100092US,United States -- Tribal Designated Statistical Area,NaN


In [ ]:
# merge data geography and data
geo_dataset = pd.merge(geography, dataset, on='GEO_ID')
geo_dataset

In [37]:
print(geography.shape)
print(dataset.shape)



(619340, 17)
(619340, 111)


In [36]:
# Merge data geography and dataset
geo_dataset = pd.merge(geography, dataset, on='GEO_ID', how='outer', indicator=True)

# Check for any unmatched rows
unmatched = geo_dataset[geo_dataset['_merge'] != 'both']
if not unmatched.empty:
    print(f"Warning: {len(unmatched)} rows did not match.")
    print("Unmatched from geography:", len(unmatched[unmatched['_merge'] == 'left_only']))
    print("Unmatched from dataset:", len(unmatched[unmatched['_merge'] == 'right_only']))

# Remove the indicator column
geo_dataset = geo_dataset.drop(columns=['_merge'])

# Display the result
print("Final merged dataset shape:", geo_dataset.shape)
geo_dataset.head()

Final merged dataset shape: (619340, 128)


,FILEID,US,REGION,DIVISION,STATE,COUNTY,COUSUB,PLACE,TRACT,BLKGRP,CBSA,CSA,METDIV,CDCURR,GEO_ID,NAME,TL_GEO_ID,B01001_E001,B01002_E001,B01003_E001,B02001_E001,B08103_E001,B08135_E001,B08301_E001,B08303_E001,B08603_E001,B08604_E001,B09001_E001,B09020_E001,B11001_E001,B11003_E001,B11007_E001,B13016_E001,B14003_E001,B15001_E001,B15011_E001,B15012_E001,B17001_E001,B17003_E001,B17006_E001,B17010_E001,B17020_E001,B18101_E001,B19001_E001,B19013_E001,B19019_E001,B19025_E001,B19037_E001,B19050_E001,B19051_E001,B19052_E001,B19053_E001,B19054_E001,B19055_E001,B19056_E001,B19057_E001,B19058_E001,B19059_E001,B19060_E001,B19061_E001,B19062_E001,B19063_E001,B19064_E001,B19065_E001,B19066_E001,B19067_E001,B19069_E001,B19070_E001,B19083_E001,B19101_E001,B19113_E001,B19119_E001,B19126_E001,B19127_E001,B19201_E001,B19313_E001,B20001_E001,B20002_E001,B20004_E001,B20005_E001,B21001_E001,B21004_E001,B22001_E001,B22002_E001,B22003_E001,B23001_E001,B24012_E001,B24032_E001,B24080_E001,B24082_E001,B24115_E001,B25001_E001,B25002_E001,B25003_E001,B25009_E001,B25010_E001,B25013_E001,B25017_E001,B25018_E001,B25019_E001,B25020_E001,B25021_E001,B25022_E001,B25024_E001,B25031_E001,B25034_E001,B25035_E001,B25036_E001,B25037_E001,B25040_E001,B25041_E001,B25042_E001,B25056_E001,B25075_E001,B25085_E001,B25086_E001,B25104_E001,B25105_E001,B25118_E001,B25119_E001,B25120_E001,B25132_E001,B25133_E001,B25134_E001,B25135_E001,B27001_E001,B27002_E001,B27003_E001
0,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100000US,United States,NaN,331097593.0,38.5,331097593.0,331097593.0,41.6,3.689928e+09,156703623.0,138386938.0,NaN,NaN,73213705.0,54737648.0,125736353.0,81432908.0,125736353.0,77210485.0,320051509.0,257883888.0,77751347.0,83932731.0,323275448.0,223101412.0,71730273.0,81432908.0,323275448.0,326147510.0,125736353.0,75149.0,75149.0,1.330706e+13,125736353.0,1.330706e+13,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,125736353.0,1.051612e+13,9.880560e+12,6.359560e+11,6.750147e+11,8.908618e+11,7.191745e+10,1.416723e+10,9.321670e+11,2.064164e+11,0.4829,81432908.0,92646.0,92646.0,92646.0,1.014083e+13,44303445.0,1.366157e+13,176037501.0,42131.0,48747.0,266411973.0,256649167.0,38374.0,125736353.0,125736353.0,125736353.0,266411973.0,45768.0,53073.0,158913204.0,53073.0,83957980.0,140943613.0,140943613.0,125736353.0,125736353.0,2.57,125736353.0,140943613.0,5.5,820090169.0,125736353.0,5.7,744990135.0,140943613.0,1268.0,140943613.0,1979.0,125736353.0,1979.0,125736353.0,140943613.0,125736353.0,44238593.0,81497760.0,1549548.0,6.192507e+11,125736353.0,1276.0,125736353.0,75149.0,1.330706e+13,125736353.0,125736353.0,125736353.0,125736353.0,326147510.0,326147510.0,326147510.0
1,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100089US,United States -- American Indian Reservation and Trust Land -- Federal,NaN,1072026.0,36.6,1072026.0,1072026.0,42.9,8.488515e+06,393272.0,359059.0,NaN,NaN,278468.0,174653.0,361167.0,243943.0,361167.0,235624.0,1032901.0,793558.0,127372.0,136976.0,1056357.0,683859.0,271310.0,243943.0,1056357.0,1062680.0,361167.0,54612.0,54612.0,2.732731e+10,361167.0,2.732731e+10,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,361167.0,1.968512e+10,1.822880e+10,1.456456e+09,1.622007e+09,2.662245e+09,3.173004e+08,6.238790e+07,2.235395e+09,7.427111e+08,0.4870,243943.0,64684.0,64684.0,64684.0,2.099607e+10,117224.0,2.884370e+10,459371.0,34205.0,38453.0,826717.0,792603.0,29624.0,361167.0,361167.0,361167.0,826717.0,37188.0,41891.0,401410.0,41891.0,NaN,450598.0,450598.0,361167.0,361167.0,2.93,361167.0,450598.0,5.2,2426839.0,361167.0,5.3,2030095.0,450598.0,763.0,450598.0,1983.0,361167.0,1983.0,361167.0,450598.0,361167.0,105263.0,255904.0,3248.0,1.048378e+09,361167.0,659.0,361167.0,54612.0,2.732731e+10,361167.0,361167.0,361167.0,361167.0,1062680.0,1062680.0,1062680.0
2,ACSSF,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100090US,United States -- American Indian Re

In [38]:
import os

# Define the path to your data folder
data_folder = 'C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data'

# Create a filename for the merged dataset
filename = 'merged_geo_dataset.csv'

# Combine the folder path and filename
full_path = os.path.join(data_folder, filename)

# Save the merged dataset to CSV
geo_dataset.to_csv(full_path, index=False)

print(f"Dataset saved to: {full_path}")

Dataset saved to: C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data\merged_geo_dataset.csv


# continue!

In [ ]:
# Create list from DF column

group_acs5 = acs5['Code'].to_list()
print(len(group_acs5))
group_acs5[:3]

In [ ]:
dataset = 'acs/acs5'
year = 2022

# get readable variable names ACS5

group_acs5_names = []
group_acs5_updated = []

for i in range(len(group_acs5)):
    try:
        #test which group is going through the loop
        # print(i)
        # print(f'Group: {group[i]}')
    
        #get the first group variable
        group_variables = ced.variables.group_variables(dataset, year, group_acs5[i])
        group_variables = group_variables[0]
        # print(group_vaariables)
    
        #get the dict for each group
        get_group = ced.variables.get_group(dataset, year, group_acs5[i])
        group_info = get_group[group_variables]
        # print(group_info)
    
        #get readable name
        label = get_group[group_variables]['label'].replace(' --','').replace(':!!', '-').replace('!!','_').replace(' ','_').replace(':','')
        concept = get_group[group_variables]['concept'].replace(' ','_')
        name = concept + " - " + label
        # print(name)
        # print(" ")
    
        group_acs5_names.append(name)
        group_acs5_updated.append(group_variables)
        
    except: 
        print(f'Group: {group_acs5[i]} FAILED')
        print(" ")
        continue
print(len(group_acs5_names))
print(len(group_acs5_updated))

group_acs5_updated
group_acs5_names

In [ ]:
variable_and_names = pd.DataFrame(list(zip(group_acs5_updated, group_acs5_names)), columns=['variables', 'names'])
variable_and_names

In [ ]:
VARIABLES = ["NAME"]
VARIABLES.extend(variable_and_names['variables'])
VARIABLES

In [ ]:
gdf_cbsa = ced.download(
    DATASET,
    YEAR,
    VARIABLES,
    metropolitan_statistical_area_micropolitan_statistical_area="*",
    with_geometry=True,
    API_KEY = api_key
)
gdf_cbsa.head()


In [ ]:
geography = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/2022/Geos20225YR.txt', sep='|')
geography